<a href="https://colab.research.google.com/github/sophielouie/beer-recommendation-system/blob/main/HybridModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Reader, Dataset, accuracy
from surprise.model_selection import train_test_split
import numpy as np
from scipy import spatial
import statistics as stats
import math
from networkx.algorithms.dag import topological_sort
import networkx as nx
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from node2vec import Node2Vec
import pickle 

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def print_test():
    print("test")

# Data Collection

In [4]:
def clean_beer_reviews():
    # storing beer review dataset
   # storing beer review dataset
    beer_reviews = pd.read_csv("beer_reviews.csv")
    # creating a unique identifier for each beer using brewery name and beer name
    beer_reviews['Unique Beer Name'] = beer_reviews['brewery_name'] + ' ' + beer_reviews['beer_name']
    # storing beer profile dataset
    beer_profile = pd.read_csv("beer_profile_and_ratings.csv")
    # columns to drop from beer reviews
    drop_cols = ['brewery_id', 'brewery_name',  'beer_name', 'beer_abv', 'beer_beerid']
    # dropping columns from beer reviews
    beer_reviews.drop(columns = drop_cols, inplace = True)
    # columns to drop from  beer profile
    drop_cols = ['Name', 'Style', 'Brewery', 'Description',
                 'Min IBU', 'Max IBU', 'Alcohol', 'review_aroma', 'review_appearance', 'review_palate',
                 'review_taste', 'review_overall', 'number_of_reviews']
    # dropping columns from beer profile
    beer_profile.drop(columns = drop_cols, inplace = True)
    # combining beer review and beer profile datasets to have profile of each beer attached to every review
    df_beer = pd.merge(beer_reviews, beer_profile, left_on = 'Unique Beer Name', right_on = 'Beer Name (Full)', how = 'inner')
    # isolating the numerical columns that need to be scaled
    need_scaling = df_beer.drop(columns = ['review_time', 'review_profilename', 'beer_style', 'Unique Beer Name', 'Beer Name (Full)'])
    # storing the informational portion of the dataset that does not need scaling
    informational = df_beer[['review_time', 'review_profilename', 'beer_style', 'Unique Beer Name', 'Beer Name (Full)']]
    # renaming beer name column
    informational.rename(columns = {'Beer Name (Full)': 'Beer Name'}, inplace = True)

    # scaling the data
    scaler = MinMaxScaler()
    scaler.fit(need_scaling)
    need_scaling = pd.DataFrame(scaler.transform(need_scaling), columns = need_scaling.columns)

    # recombining the informational data and scaled data
    df = pd.concat([informational, need_scaling], axis = 1)
    return df

In [5]:
df = clean_beer_reviews()

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [34]:
print(df)

                                                                  review_time  \
review_profilename Unique Beer Name                                             
fodeeoz            Amstel Brouwerij B. V. Amstel Light             1010963392   
jdhilt             Amstel Brouwerij B. V. Amstel Light             1010861086   
xXTequila          Amstel Brouwerij B. V. Amstel Light             1002109880   
Brent              Amstel Brouwerij B. V. Amstel Light              988202869   
brownbeer          Amstel Brouwerij B. V. Amstel Light             1264904623   
...                                                                       ...   
treehugger02010    The Defiant Brewing Company Muddy Creek Lager   1245807636   
njthebestofme      The Defiant Brewing Company Muddy Creek Lager   1236320811   
barleywinebrewer   The Defiant Brewing Company Muddy Creek Lager   1221010852   
maddogruss         The Defiant Brewing Company Muddy Creek Lager   1175747094   
ArrogantB          The Defia

# Hybrid Model

Conceptualize the hybrid model

In [6]:
df.set_index(['review_profilename', 'Unique Beer Name'], inplace = True)

#Hybrid Testing

We need to select users to isolate for our test set. The criteria for these users is that they should have reviewed enough beers that there is some likelihood that they would have tried something that has been recommended to them. Starting with binary cumulative gain, we will assess the performance of our recommendation system by determining if the recommended beers have been tried by the user or exceeded a threshold.

- recommend x beers to 10 users
- traverse the recommended beer list, see how many have been rated highly
- compare the accuracy ratings of the three test users
- determine threshold for positive review by finding percentiles of ratings (top 1/4 review would be considered a successful recommendation)

## Choosing users for our test set

In [7]:
freq_reviews = df.groupby('review_profilename').count().sort_values(by=['review_time'], ascending = False)
freq_reviews.rename(columns={'review_time': 'Number of Reviews'}, inplace = True)
freq_reviews['Number of Reviews']

review_profilename
BuckeyeNation     1505
mikesgroove       1422
northyorksammy    1348
brentk56          1225
BEERchitect       1201
                  ... 
cheapdark            1
cheath               1
chefguy              1
chefmonty            1
zzajjber             1
Name: Number of Reviews, Length: 26631, dtype: int64

The ten users with the most beer reviews

In [8]:
test_users = freq_reviews.index[:10]

In [48]:
def hybrid(user, beer, n_recs, df, svd_model):
    print(df)
    values = ['review_aroma', 'review_appearance', 'review_palate', 'review_taste', 'ABV', 'Astringency', 'Body', 'Bitter', 'Sweet', 'Sour', 'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty']

    target_beer_vector = df.loc[(user, beer)][values]

    # list of top 50 similar beers
    similar = []
    for beer_name in df["Beer Name"].unique():
        beer_vectors = df.loc[df["Beer Name"] == beer_name][values]
        cos_sim = list()
        for beer_vector in np.array(beer_vectors):
            cos_sim.append(1 - spatial.distance.cosine(beer_vector, target_beer_vector))
        similar.append((beer_name, stats.mean(cos_sim)))

    # sort in decreasing order
    similar = sorted(similar, key = lambda x: x[1], reverse = True)
    sim = similar[1:50]

    # get metadata for each of 50 similar beers
    beer_idx = [i[0] for i in sim]
    # we are trying to make a list of the top 50 similar beers and its metadata
    # there are multiple reviews for a single beer by different users, so we will average review_overall for each similar beer --> groupby and average
    beers = pd.DataFrame()

    for idx in beer_idx:
        average_rating = df[df['Beer Name'] == idx].groupby('Beer Name').mean()[['review_overall']]
        beers = pd.concat([beers, average_rating])
  
    beers = beers.reset_index()

    # create an "est" column and apply SVD.predict() to each book
    # predict using the svd_model
    beers['est'] = beers.apply(lambda x: svd_model.predict(user, x['Beer Name'], x['review_overall']).est, axis = 1)

    # sort predictions in decreasing order
    beers = beers.sort_values(by = 'est', ascending = False)

    # return top n recommendations
    return beers[:n_recs]

##Find threshold for what makes a good rating

In [10]:
len(df.loc[df.review_overall >= .875])

219603

In [11]:
THRESHOLD = 0.875

Only 219,603 / 744,251 reviews (29.5%) were given a score of .875 or greater, so this will be our threshold for a successful recommendation

## Remove all ratings by the test users from the training set except for the beer they rated the highest

In [12]:
def create_train_test_split(df, test_users, frac_rem=1):
    #frac_rem: fraction of each user in test set to remain in train set
    # train_set is copy of df --> df has review_profilename and Unique Beer Name as indicies
    # we are preserving user information by reseting indicies, so when we .loc[] we still have access to user and beer info
    train_set = df.copy().reset_index()
    test_parameters = []
    test_set = pd.DataFrame(columns = train_set.columns)


    for user in test_users: 
        # all reviews for a the test user
        user_reviews = train_set.loc[train_set.review_profilename == user]

        # sorted reviews
        sorted_user_reviews = user_reviews.sort_values(by = 'review_overall', ascending = False)

        # store highest reviewed beer
        highest_reviewed_beer = sorted_user_reviews.iloc[0]
        test_parameters.append((user, highest_reviewed_beer["Beer Name"]))

        # removing highest reviewed beer from user_reviews so that it remains in the train set
        user_reviews.drop(highest_reviewed_beer.name, axis = 0, inplace = True)

        # calculating the last index to remove from the train set
        last_idx = int((len(user_reviews) - 1) * frac_rem)

        # concatenate the removed user-beer pairs to test set
        test_set = pd.concat([test_set, user_reviews.iloc[0:last_idx]])

        # remove all beers from training set
        train_set.drop(user_reviews.iloc[0:last_idx].index, axis = 0, inplace = True)

    return train_set, test_set, test_parameters

In [13]:
train_set, test_set, test_parameters = create_train_test_split(df, test_users, .5)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
def train_svd(train_set, test_set):
    # creating and training SVD on train_set
    reader = Reader()

    # train_set = train_set.reset_index()
    # test_set = test_set.reset_index()

    test_data = Dataset.load_from_df(test_set[['review_profilename', 'Beer Name', 'review_overall']], reader)
    train_data = Dataset.load_from_df(train_set[['review_profilename', 'Beer Name', 'review_overall']], reader)

    # NEED TO TURN OUR TRAIN_SET INTO surprise TRAINSET TYPE
    surprise_train_set = train_data.build_full_trainset()

    # train
    svd = SVD()
    svd.fit(surprise_train_set)
    return svd
# NEED TO FIGURE OUT IF SVD USES USER INFORMATION FOR CALCULATIONS --> can this model perform well without significant historical review data

In [24]:
svd = train_svd(train_set, test_set)
# Save to pkl file
filehandler = open('svd.pkl', 'w') 
pickle.dump(svd, filehandler)

KeyError: "['review_profilename'] not in index"

In [51]:
def hybrid_model(user, beer, n_recs, test_frac = 1):
    # Collecting and cleaning the dataset
    df = clean_beer_reviews()
    df.set_index(['review_profilename', 'Unique Beer Name'], inplace = True)
    # Load svd pkl file
    filehandler = open('svd.pkl', 'rb') 
    svd = pickle.load(filehandler)
    return hybrid(user, beer, n_recs, df, svd)

In [52]:
hybrid_model('Brent', 'Amstel Brouwerij B. V. Amstel Light', 10)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


                                                                  review_time  \
review_profilename Unique Beer Name                                             
fodeeoz            Amstel Brouwerij B. V. Amstel Light             1010963392   
jdhilt             Amstel Brouwerij B. V. Amstel Light             1010861086   
xXTequila          Amstel Brouwerij B. V. Amstel Light             1002109880   
Brent              Amstel Brouwerij B. V. Amstel Light              988202869   
brownbeer          Amstel Brouwerij B. V. Amstel Light             1264904623   
...                                                                       ...   
treehugger02010    The Defiant Brewing Company Muddy Creek Lager   1245807636   
njthebestofme      The Defiant Brewing Company Muddy Creek Lager   1236320811   
barleywinebrewer   The Defiant Brewing Company Muddy Creek Lager   1221010852   
maddogruss         The Defiant Brewing Company Muddy Creek Lager   1175747094   
ArrogantB          The Defia

<ipython-input-48-61116dc6554f>:5: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_beer_vector = df.loc[(user, beer)][values]


,Beer Name,review_overall,est
0,Flensburger Brauerei GmbH Und Co. KG Flensburg...,0.660377,1
25,Summit Brewing Company Summit Maibock,0.668011,1
27,AC Golden Brewing Company Colorado Native Lager,0.707237,1
28,Strangford Lough Brewing Company Ltd St. Patri...,0.674419,1
29,Wellington County Brewery Trailhead Lager,0.523256,1
30,Kulmbacher Brauerei AG Kulmbacher Edelherb Pre...,0.746711,1
31,Žatecký Pivovar Žatec,0.685000,1
32,Brauerei Pinkus Mueller Organic Ur Pils,0.726163,1
33,New Glarus Brewing Company Totally Naked,0.684152,1
34,Samuel Smith Old Brewery (Tadcaster) Samuel Sm...,0.742399,1


Separate Testing Function

In [18]:
# setting train_set index to be compatible with hybrid()
train_set.set_index(["review_profilename", "Unique Beer Name"], inplace = True)

In [19]:
def compile_rec_list(n_recs, test_parameters):
    rec_dict = {}
    # iterate through test users, get X recommendations for their only beer review, compare recommendations to actual ratings stored in test_set using DCG
    for user, beer in test_parameters[0:5]:
        recommendations = hybrid(user, beer, n_recs, train_set, svd)
        rec_dict[user] = list(recommendations["Beer Name"])

    return rec_dict

In [20]:
# pass test_parameters in as parameter !!
recs = compile_rec_list(10, test_parameters)

df
                                                                  review_time  \
review_profilename Unique Beer Name                                             
fodeeoz            Amstel Brouwerij B. V. Amstel Light             1010963392   
jdhilt             Amstel Brouwerij B. V. Amstel Light             1010861086   
xXTequila          Amstel Brouwerij B. V. Amstel Light             1002109880   
Brent              Amstel Brouwerij B. V. Amstel Light              988202869   
brownbeer          Amstel Brouwerij B. V. Amstel Light             1264904623   
...                                                                       ...   
treehugger02010    The Defiant Brewing Company Muddy Creek Lager   1245807636   
njthebestofme      The Defiant Brewing Company Muddy Creek Lager   1236320811   
barleywinebrewer   The Defiant Brewing Company Muddy Creek Lager   1221010852   
maddogruss         The Defiant Brewing Company Muddy Creek Lager   1175747094   
ArrogantB          The De

<ipython-input-9-603d08ff2fe4>:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_beer_vector = df.loc[(user, beer)][values]


49
                                            Beer Name  review_overall  est
0         Monday Night Brewing Drafty Kilt Scotch Ale        0.791667    1
25             Rahr & Sons Brewing Company Angry Goat        0.729167    1
27                   Crabtree Brewing Company Braggot        0.675000    1
28            Lonerider Brewing Sweet Josie Brown Ale        0.713415    1
29  Lagerheads Smokehouse & Brewing Co. Barnburner...        0.593750    1
30             Sun King Brewing Company Wee Mac Brown        0.666667    1
31               Avery Brewing Company Old Jubilation        0.709993    1
32   U Fleků Pivovaru a Restauraci U Fleku Dark Lager        0.883621    1
33     Three Floyds Brewing Co. & Brewpub Region Riot        0.656250    1
34  Moat Mountain Smoke House & Brewing Co. Moat B...        0.817308    1
35  Bierbrouwerij De Koningshoeven B.V. La Trappe ...        0.744565    1
36            Flying Fish Brewing Company OktoberFish        0.672727    1
37  Saint Arnold Brewi

<ipython-input-9-603d08ff2fe4>:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_beer_vector = df.loc[(user, beer)][values]


49
                                            Beer Name  review_overall  est
0           Spaten-Franziskaner-Bräu Spaten Optimator        0.723642    1
25           Portsmouth Brewery Portsmouth Killer Bee        0.857143    1
27  Oskar Blues Grill & Brew Old Chub - Scottish S...        0.738977    1
28                    Deschutes Brewery Mirror Mirror        0.774123    1
29  Utah Brewers Cooperative Wasatch Devastator Do...        0.667279    1
30                  Capital Brewery Blonde Doppelbock        0.723958    1
31               Jackie O's Pub & Brewery Chomolungma        0.715909    1
32  Brauerei Hofstetten Krammer GmbH & Co. KG Gran...        0.675847    1
33  Brauerei Hofstetten Krammer GmbH & Co. KG Gran...        0.750000    1
34  Bierbrouwerij De Koningshoeven B.V. La Trappe ...        0.744565    1
35           Abbaye de Leffe S.A. Leffe Brune / Brown        0.707143    1
36        Lagunitas Brewing Company Hairy Eyeball Ale        0.693733    1
37  Kuhnhenn Brewing C

<ipython-input-9-603d08ff2fe4>:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_beer_vector = df.loc[(user, beer)][values]


49
                                            Beer Name  review_overall  est
0      Brasserie de Rochefort Trappistes Rochefort 10        0.834741    1
25               Sprecher Brewing Company Dopple Bock        0.743873    1
27   Fat Heads Brewery & Saloon Alpen Glow Weizenbock        0.750000    1
28        Dry Dock Brewing Co. Bligh's Barleywine Ale        0.862500    1
29               De Struise Brouwers Pannepot Reserva        0.810078    1
30        Victory Brewing Company Moonglow Weizenbock        0.742546    1
31            Russian River Brewing Company Salvation        0.767333    1
32              Tröegs Brewing Company Tröegs Mad Elf        0.704254    1
33  The Duck-Rabbit Craft Brewery Duck-Rabbit Wee ...        0.693441    1
34                  Brouwerij Contreras Valeir Donker        0.700000    1
35  Boston Beer Company (Samuel Adams) Samuel Adam...        0.705458    1
36                       De Struise Brouwers Pannepot        0.817366    1
37        AleSmith Bre

<ipython-input-9-603d08ff2fe4>:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  target_beer_vector = df.loc[(user, beer)][values]


KeyboardInterrupt: 

In [ ]:
def calc_score(metric, rec_dict, n_recs, test):
    # test is a dataframe consisting of the user reviews to be used to determine relevancy
    score = 0
    index = 1
    num_rel = 0
    for user, recs in rec_dict.items():

        for rec in recs:
            
            # we need to include Raise And Catch Exception when beer is not in test set
            row = test.loc[(test.review_profilename == user) & (test["Unique Beer Name"] == rec)]

            # if rec in test_set and above threshold
            if len(row) > 0:
                if row["review_overall"].iloc[0] >= THRESHOLD:
                    num_rel += 1

                    if metric == "CG":
                        score += 1

                    elif metric in ["DCG", 'NDCG']:
                        score += (1 / math.log2(index + 1))

                    elif metric == "MAP":
                        score += num_rel / index

                    else:
                        raise Exception("Metric type provided is not valid.")

            if metric == 'NDCG':
                ideal = 0
                for num in range(num_rel):
                    # num starts at 0, so we add two to mimic the starting index of 1\
                    ideal += (1 / math.log2(num + 2))
    
                score = score / ideal

    if metric == "MAP":
        score = score / n_recs

    index += 1
    print(f"{user} : {score}")

In [ ]:
calc_score("CG", recs, 10, test_set)